<a href="https://colab.research.google.com/github/sei00f/python-scraping-kadai/blob/main/test1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium

import math
import time
from datetime import datetime
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import MoveTargetOutOfBoundsException, NoSuchElementException

# ヘッドレスモードで起動するためのオプションを設定
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--window-size=1920,1080')

# WebDriverのインスタンスを作成
chrome_driver = webdriver.Chrome(options=chrome_options)

# 日経新聞へアクセス※3ヶ月
chrome_driver.get('https://www.nikkei.com/markets/worldidx/chart/nk225/?type=6month')


# 株価が表示されるまで待つ
wait = WebDriverWait(chrome_driver, 30)
wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'div.m-trend_chart_tab')))
#MAIN_CONTENTS > div.m-block.m-trend_chart > div.m-trend_chart_tab

# グラフの要素を探す
graph_element = chrome_driver.find_element(By.CSS_SELECTOR, 'rect')

# ActionChainsのインスタンスを作成
action = ActionChains(chrome_driver)
graph_width = graph_element.size['width']
graph_height = graph_element.size['height']
# sizeが奇数の場合小数点を切り上げる
offset_x = math.ceil(graph_width / 2)
offset_y = math.ceil(graph_height / 2)
print(f"width: {graph_width}, offset:{offset_x}")  # 偶数の確認

# グラフの要素にマウスを移動させる
action.move_to_element(graph_element).perform()
action.move_by_offset(offset_x, 0).perform()


def get_stock_values(chrome_driver, graph_element):
    action = ActionChains(chrome_driver)
    graph_width = graph_element.size['width']
    graph_height = graph_element.size['height']

    stock_data = []
    for _ in 5:  # グラフの幅に対してループ
        extract_stock_data(stock_data)
        try:
            # 1px左に移動
            action = ActionChains(chrome_driver)
            action.move_to_element_with_offset(graph_element, graph_width - _, graph_height / 2).perform()
        except MoveTargetOutOfBoundsException as e:
            print(f"Error moving left by 1px: {e}")
            break
        time.sleep(0.1)  # 少し待つことでデータが更新されるのを待つ
print(stock_data)


# # HTMLから株価データを抽出する関数
# def extract_stock_data(stock_data):
#     soup = BeautifulSoup(chrome_driver.page_source, 'html.parser')
#     rows = soup.select('tbody tr')

#     for row in rows:
#         td_elements = row.find_all('td')
#         if len(td_elements) < 5:
#             continue
#         # 日付のフォーマット変更
#         date_text = td_elements[0].text.strip()
#         # 文字列を日付オブジェクトに変換し、フォーマットを変更
#         date_obj = datetime.strptime(date_text, "%Y/%m/%d")
#         formatted_date = date_obj.strftime("%Y-%m-%d")
#         open_price = td_elements[1].text.strip()
#         high_price = td_elements[2].text.strip()
#         low_price = td_elements[3].text.strip()
#         close_price = td_elements[4].text.strip()

#         stock_data.append([formatted_date, open_price, high_price, low_price, close_price])

# # 株価データを取得
# stock_data = get_stock_values(chrome_driver, graph_element)
